In [1]:
!wget https://raw.githubusercontent.com/soydz/ia-saberpro-prediction/main/data/train.zip
!wget https://raw.githubusercontent.com/soydz/ia-saberpro-prediction/main/data/test.zip
!unzip train.zip
!unzip test.zip

--2025-11-26 15:07:13--  https://raw.githubusercontent.com/soydz/ia-saberpro-prediction/main/data/train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19229940 (18M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  18.34M  --.-KB/s    in 0.05s   

2025-11-26 15:07:15 (366 MB/s) - ‘train.zip’ saved [19229940/19229940]

--2025-11-26 15:07:15--  https://raw.githubusercontent.com/soydz/ia-saberpro-prediction/main/data/test.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7966338 (7.6M) 

In [20]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import unicodedata
import re


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print('train', train.shape)
print('test', test.shape)

train (692500, 21)
test (296786, 20)


### Preprocesamiento

#### Funciones

In [21]:
def clean_data_base(data):
  data.drop(columns=['F_TIENEINTERNET.1', 'ID', 'PERIODO_ACADEMICO', 'E_PRIVADO_LIBERTAD'], inplace=True)

  data['E_PRGM_ACADEMICO'] = data['E_PRGM_ACADEMICO'].apply(lambda x: unicodedata.normalize('NFKD', str(x)).encode('ASCII', 'ignore').decode('ASCII'))
  data['E_PRGM_ACADEMICO'] = data['E_PRGM_ACADEMICO'].str.replace(r'\s+', ' ', regex=True).str.strip()

  reemplazos = {
    r'\bADMINISTRACIN\b': 'ADMINISTRACION',
    r'\bLOGSTICA\b': 'LOGISTICA',
    r'\bPBLICA\b': 'PUBLICA',
    r'\bGESTIN\b': 'GESTION',
    r'\bQUMICA\b': 'QUIMICA',
    r'\bFARMACUTICA\b': 'FARMACEUTICA',
    r'\bGASTRONOMA\b': 'GASTRONOMIA',
    r'\bPSICOSOCIALES\b': 'PSICOSOCIALES',
  }
  data['E_PRGM_ACADEMICO']= data['E_PRGM_ACADEMICO'].replace(reemplazos, regex=True)

  reemplazos = {
    'MERCADEO':'ECONOMIA',
    'MERCADEO NACIONAL E INTERNACIONAL':'ECONOMIA',
    'MERCADEO Y VENTAS':'ECONOMIA',
    'PROFESIONAL EN MERCADEO':'ECONOMIA',
    'PROFESIONAL EN MERCADEO EMPRESARIAL':'ECONOMIA'
  }
  data['E_PRGM_ACADEMICO']= data['E_PRGM_ACADEMICO'].replace(reemplazos)

  return data

Se agrupan los programas academicos segun la clasificación del SNIES.
El SNIES es el Sistema Nacional de Información de la Educación Superior de Colombia.

Definición de áreas SNIES (agrupadas en 6 grupos)

In [22]:
snies_areas = {
  'SALUD': [
    r'\b(microbiolog[iaí]|bioan[áa]lisis|gerontolog[iaí]|'
    r'seguridad\s+y\s+salud\s+(en\s+el\s+trabajo|para\s+el\s+trabajo))\b',
    r'\b(medicina|enfermer[iaí]|odontolog[iaí]|fisioterapia|nutrici[óo]n|'
    r'optometr[iaí]|bacteriolog[iaí]|fonoaudiolog[iaí]|psicolog[iaí]\s+cl[ií]nica|'
    r'terapia\s+(ocupacional|cardiorrespiratoria|respiratoria)|instrumentaci[óo]n\s+quir[uú]rgica|'
    r'ciencias\s+de\s+la\s+salud|salud\s+p[úu]blica|salud\s+ocupacional|'
    r'bioquimica\s+cl[ií]nica|microbiolog[iaí]\s+y\s+bioan[áa]lisis|'
    r'farmacia|gerontolog[iaí]|seguridad\s+y\s+salud\s+en\s+el\s+trabajo)\b',
    r'\bBACTERIOLOGIA\b',
    r'\bFONOAUDIOLOGIA\b',
    r'\bODONTOLOGIA\b',
    r'\bOPTOMETRIA\b',
    r'\bENFERMERIA\b',
    r'\bMICROBIOLOG[ÍI]A\b',
    r'\bGERONTOLOG[ÍI]A\b',
    r'TERAPIAS PSICOSOCIALES'
  ],

  'INGENIERIA': [
    r'\bINGENIERIA\s+(ADMINISTRATIVA|AERONAUTICA|COMERCIAL|DE\s+MERCADOS|'
    r'DE\s+PETROLEO\s+Y\s+GAS|DE\s+PROCESOS(\s+INDUSTRIALES)?|'
    r'DE\s+PRODUCCION(\s+ACUICOLA)?|DE\s+PRODUCTIVIDAD\s+Y\s+CALIDAD|'
    r'DE\s+TELECOMUNICACIONES|EN\s+(ENERG[IAÍ]|HIGIENE\s+Y\s+SEGURIDAD\s+OCUPACIONAL|'
    r'MULTIMEDIA|PROCESOS\s+INDUSTRIALES|PRODUCCION\s+ACUICOLA|'
    r'SEGURIDAD\s+Y\s+SALUD\s+PARA\s+EL\s+TRABAJO|SOFTWARE|TELECOMUNICACIONES)|'
    r'MULTIMEDIA|TOPOGRAFICA)\b',
    r'\b(SISTEMAS|COMPUTACION|INFORMATICA|TELEMATICA|'
    r'TELECOMUNICACIONES|ELECTRONICA|ELECTRICA|INDUSTRIAL|'
    r'MECANICA|QUIMICA|AMBIENTAL|CIVIL|BIOMEDICA|SOFTWARE|'
    r'GEOLOGICA|METALURGICA|ALIMENTOS|AGRONOMICA|'
    r'MINAS|PETROLEOS|AGRICOLA|AGROECOLOGICA|AGROFORESTAL|'
    r'AGROINDUSTRIAL|BIOLOGICA|BIOQUIMICA|BIOTECNOLOGICA|'
    r'DISENO\s+DE\s+PRODUCTO|MATERIALES|SONIDO|TRANSPORTE|'
    r'ELECTROMECANICA|MECATRONICA|NANOTECNOLOGIA|TOPOGRAFIA|FORESTAL|'
    r'NAVAL|PECUARIA|PESQUERA|SANITARIA|FINANCIERA|'
    r'CONTROL|AUTOMATIZACION|INSTRUMENTACION)\b',
    r'\b(ARQUITECTURA|CONSTRUCCION|CONSTRUCCIONES\s+CIVILES)\b',
    r'\bINGENIERA|INGENIERIA\b',
    r'URBANISMO',
  ],

  'SOCIALES': [
    r'\b(derecho|jurisprudencia|psicolog[iaí](?!\s+(cl[ií]nica|ocupacional))|'
    r'comunicaci[óo]n|periodismo|relaciones\s+(internacionales|econ[óo]micas\s+internacionales)|'
    r'ciencia\s+pol[ií]tica|trabajo\s+social|antropolog[iaí]|sociolog[iaí]|'
    r'gobierno\s+y\s+asuntos\s+p[úu]blicos|planeaci[óo]n\s+y\s+desarrollo\s+social|'
    r'estudios\s+(pol[ií]ticos|literarios|musicales|sociculturales)|'
    r'criminolog[iaí]|criminal[íi]stica|investigaci[óo]n\s+criminal|'
    r'ling[uü]istica|lenguajes)\b',
    r'\bANTROPOLOGIA\b',
    r'\bSOCIOLOGIA\b',
    r'\bCIENCIAS POLITICAS\b',
    r'\bCOMUNICACIONES\b',
    r'\bLINGUISTICA\b',
    r'\bLENGUAJES Y ESTUDIOS SOCIOCULTURALES\b',
    r'\bPSICOLOGIA\b',
    r'\bPROFESIONAL EN PSICOLOGIA\b',
    r'ARQUEOLOGIA',
    r'DESARROLLO TERRITORIAL',
    r'\b(GESTION\s+CULTURAL|GESTION\s+DEPORTIVA|GESTION\s+Y\s+DESARROLLO\s+URBANOS)\b',
    r'PEDAGOGIA'
  ],

  'EDUCACION': [
    r'\b(licenciatura|pedagog[iaí]|educaci[óo]n|docencia|enseñanza|'
    r'did[áa]ctica|formaci[óo]n\s+docente|desarrollo\s+familiar)\b',
  ],

  'ECONOMIA': [
    r'MERCADEO',
    r'\b(?:MERCADEO(?: NACIONAL E INTERNACIONAL| EMPRESARIAL)?|PROFESIONAL EN MERCADEO(?: EMPRESARIAL)?)\b',
    r'\b(?:MERCADEO|PROFESIONAL\s+EN\s+MERCADEO)(?:\s+(?:NACIONAL\s+E\s+INTERNACIONAL|EMPRESARIAL))?\b',
    r'\b(administraci[óo]n|contadur[iaí]|econom[iaí]|finanzas|mercado[lt]og[íi]a|'
    r'marketing|negocios|gerencia|log[ií]stica|turismo|gesti[óo]n\s+(empresarial|'
    r'cultural\s+y\s+comunicativa|del\s+dise[ñn]o|de\s+la\s+seguridad|'
    r'de\s+mercados|financiera|urbana|territorial)|comercio\s+(exterior|internacional)|'
    r'administraci[óo]n\s+tur[íi]stica\s+y\s+hotelera|'
    r'contadur[iaí]\s+(publica|internacional)|mercadeo|econom[iaí]a)\b',
    r'\b(?:MERCADEO|MARKETING|PUBLICIDAD)(?:\s+(?:INTERNACIONAL|NACIONAL|EMPRESARIAL|Y\s+(?:PUBLICIDAD|MERCADEO|VENTAS|INTERNACIONAL)|E\s+INTERNACIONAL))?\b',
    r'\bPROFESIONAL\s+EN\s+(?:MERCADEO|MARKETING)(?:\s+(?:EMPRESARIAL|Y\s+PUBLICIDAD))?\b',
    r'\b(?:ADMINISTRACI[ÓO]N|CONTADUR[ÍI]A|ECONOM[ÍI]A|FINANZAS|'
    r'GERENCIA|LOG[ÍI]STICA|TURISMO|NEGOCIOS|COMERCIO)\b',
  ],

  'CIENCIAS': [
    r'\b(matem[áa]tica|f[íi]sica|qu[íi]mica|biolog[iaí]|geolog[iaí]|'
    r'estad[ií]stica|ciencias\s+b[áa]sicas|ecolog[iaí]|'
    r'ciencias\s+(ambientales|biol[óo]gicas|del\s+mar|n[áa]uticas|'
    r'militares\s+(aeron[áa]uticas|navales)?)|geograf[iaí]|'
    r'astrof[íi]sica|geociencias|bioingenier[iaí]a|'
    r'biotecnolog[iaí]a|bioquimica|deporte|recreaci[óo]n|'
    r'entrenamiento\s+deportivo)\b',
    r'\bBIOLOGIA\b',
    r'\bECOLOGIA\b',
    r'\bMATEMATICAS\b',
    r'\bGEOGRAFIA\b',
    r'\bGEOLOGIA\b',
    r'\bDEPORTE\b',
    r'\bRECREACION\b',
    r'\bENTRENAMIENTO DEPORTIVO\b',
    r'\bPROFESIONAL EN DEPORTE\b',
    r'\bPROFESIONAL EN ENTRENAMIENTO DEPORTIVO\b',
    r'\bCIENCIAS DEL DEPORTE\b',
    r'\bCIENCIAS DEL DEPORTE Y LA RECREACION\b',
    r'\bCIENCIAS MILITARES\b',
    r'\bCIENCIAS NAVALES PARA OFICIALES DE INFANTERIA DE MARINA\b',
    r'\bCIENCIAS NAVALES PARA OFICIALES NAVALES\b',
    r'ASTRONOMIA',
    r'OCEANOGRAFIA'
  ],

  'ARTES_HUMANIDADES': [
    r'\b(filosof[iaí]|historia|literatura|lenguas|traducci[óo]n|'
    r'arte|danza|dram[áa]tico|cine|audiovisuales|televisi[óo]n|'
    r'radio|producci[óo]n\s+(cinematogr[áa]fica|audiovisual|musical)|'
    r'dise[ñn]o\s+(gr[áa]fico|industrial|interior|de\s+modas|'
    r'de\s+espacios|digital|interactivo|visual|multimedia|'
    r'de\s+vestuario|escenogr[áa]fico)|gastronom[iaí]a|'
    r'culinaria|teolog[iaí]|musicolog[iaí]|interpretaci[óo]n\s+musical|'
    r'fotograf[iaí]a|bibliotecolog[iaí]a|archiv[íi]stica|'
    r'creaci[óo]n\s+literaria|espanol|filologia|'
    r'musica|formaci[óo]n\s+musical|estudios\s+(literarios|musicales|culturales))\b',
    r'\bARTES\b',
    r'\bMUSICA\b',
    r'\bFILOSOFIA\b',
    r'\bFILOSOFIA Y HUMANIDADES\b',
    r'\bFILOSOFIA Y LETRAS\b',
    r'\bESPANOL Y FILOLOGIA CLASICA\b',
    r'\bFILOLOGIA E IDIOMAS\b',
    r'\bFILOLOGIA HISPANICA\b',
    r'\bCIENCIAS BIBLICAS\b',
    r'\bTEOLOGIA\b',
    r'\bDISENO\b',
    r'\bDISENO DE MEDIOS INTERACTIVOS\b',
    r'\bDISENO Y GESTION DE LA MODA\b',
    r'\bDISENO Y PRODUCCION DE MODA\b',
    r'\bCREACION LITERARIA\b',
    r'\bESTUDIOS Y GESTION CULTURAL\b',
    r'\bCIENCIAS\s+DE\s+LA\s+INFORMACION\s+Y\s+LA\s+DOCUMENTACION\b',
    r'ANIMACION',
    r'TEATRO',
    r'DIRECCION DE BANDA',
    r'CONSERVACION Y RESTAURACION DE BIENES MUEBLES',
    r'CROSSMEDIA',
    r'NARRATIVAS DIGITALES',
  ],

  'AGROPECUARIO': [
    r'\b(agronom[iaí]|agroecolog[iaí]a|agroforestal|agroindustrial|'
    r'agropecuaria|zootecnia|acuicultura|pesquera|'
    r'producci[óo]n\s+agroindustrial|agronegocios|'
    r'profesional\s+en\s+(agroindustria|agronegocios))\b',
    r'\bAGRONOMIA\b',
    r'\bAGRONOMIA DEL TROPICO HUMEDO\b',
  ]
}


# Función para mapear programa a área
def mapper_prgm_academico_to_snies(programa):
  if not isinstance(programa, str):
      return 'OTRO'

  # Corrección de errores tipográficos frecuentes
  p = programa.upper().strip()
  p = re.sub(r'[^\w\s]', ' ', p)  # Elimina puntuación
  p = re.sub(r'\s+', ' ', p)     # Elimina espacios múltiples

  # Corección de errores de digitación del dataset
  p = re.sub(r'\bADMINSITRACION\b', 'ADMINISTRACION', p)
  p = re.sub(r'\bINTRUMENTACION\b', 'INSTRUMENTACION', p)
  p = re.sub(r'\bCOMUNICACIN\b', 'COMUNICACION', p)
  p = re.sub(r'\bMERCADEO\b', 'MERCADOTECNIA', p)
  p = re.sub(r'\bCOMUNICACIN\b', 'COMUNICACION', p)
  p = re.sub(r'\bPSICOLOGIA\b', 'PSICOLOGIA', p)

  # Busca en cada área
  for area, patrones in snies_areas.items():
      for patron in patrones:
          if re.search(patron, p, re.IGNORECASE):
              return area
  return 'OTRO'

In [23]:
def clean_prgm_departamento_to_region(data):
  region_map = {
    # Caribe
    'ATLANTICO': 'CARIBE', 'BOLIVAR': 'CARIBE', 'SUCRE': 'CARIBE', 'MAGDALENA': 'CARIBE',
    'LA GUAJIRA': 'CARIBE', 'CESAR': 'CARIBE', 'SAN ANDRES': 'CARIBE',

    # Andina
    'BOGOTÁ': 'ANDINA', 'ANTIOQUIA': 'ANDINA', 'HUILA': 'ANDINA', 'CUNDINAMARCA': 'ANDINA',
    'TOLIMA': 'ANDINA', 'VALLE': 'ANDINA', 'QUINDIO': 'ANDINA', 'RISARALDA': 'ANDINA',
    'CORDOBA': 'ANDINA', 'BOYACA': 'ANDINA', 'NARIÑO': 'ANDINA', 'CALDAS': 'ANDINA',
    'SANTANDER': 'ANDINA', 'NORTE SANTANDER': 'ANDINA',

    # Pacífica
    'CAUCA': 'PACIFICA', 'CHOCO': 'PACIFICA', 'NARIÑO': 'PACIFICA',

    # Orinoquía
    'META': 'ORINOQUIA', 'CASANARE': 'ORINOQUIA', 'ARAUCA': 'ORINOQUIA', 'GUAVIARE': 'ORINOQUIA',

    # Amazónica
    'AMAZONAS': 'AMAZONICA', 'CAQUETA': 'AMAZONICA', 'PUTUMAYO': 'AMAZONICA',
    'VAUPES': 'AMAZONICA'
  }

  data['E_PRGM_DEPARTAMENTO'] = data['E_PRGM_DEPARTAMENTO'].map(region_map).fillna('OTRO')

  return data

In [24]:
def clean_nulos(data):
  # Elimina filas que no tienen al menos el 70% de datos validos
  data.dropna(axis=0, thresh=int(len(data.columns) * 0.7), inplace=True)
  data.dropna(subset=['E_VALORMATRICULAUNIVERSIDAD', 'E_PAGOMATRICULAPROPIO'], inplace=True)

  imputer = SimpleImputer(strategy='most_frequent')
  data['F_ESTRATOVIVIENDA'] = imputer.fit_transform(data[['F_ESTRATOVIVIENDA']])[:, 0]
  data['F_TIENEINTERNET'] = imputer.fit_transform(data[['F_TIENEINTERNET']])[:, 0]
  data['F_TIENECOMPUTADOR'] = imputer.fit_transform(data[['F_TIENECOMPUTADOR']])[:, 0]
  data['F_TIENELAVADORA'] = imputer.fit_transform(data[['F_TIENELAVADORA']])[:, 0]
  data['F_TIENEAUTOMOVIL'] = imputer.fit_transform(data[['F_TIENEAUTOMOVIL']])[:, 0]

  data['F_EDUCACIONPADRE'] = data.groupby('F_ESTRATOVIVIENDA')['F_EDUCACIONPADRE'].transform(lambda x: x.fillna(x.mode()[0]))
  data['F_EDUCACIONMADRE'] = data.groupby('F_ESTRATOVIVIENDA')['F_EDUCACIONMADRE'].transform(lambda x: x.fillna(x.mode()[0]))

  data['E_HORASSEMANATRABAJA'] = data['E_HORASSEMANATRABAJA'].map({'0': 0, 'Menos de 10 horas': 9, 'Entre 11 y 20 horas': 15.5, 'Entre 21 y 30 horas': 25.5, 'Más de 30 horas': 31})
  data['E_HORASSEMANATRABAJA'] = data['E_HORASSEMANATRABAJA'].fillna((data['E_HORASSEMANATRABAJA'].mean()).round(3))

  return data

In [25]:
def categorical_to_numeric(data):
  data = pd.get_dummies(data, columns=['E_VALORMATRICULAUNIVERSIDAD'], dtype=np.uint8)
  data.drop(columns=['E_VALORMATRICULAUNIVERSIDAD_No pagó matrícula'], inplace=True)

  data = pd.get_dummies(data, columns=['F_ESTRATOVIVIENDA'], dtype=np.uint8)
  data.drop(columns=['F_ESTRATOVIVIENDA_Sin Estrato'], inplace=True)
  data.columns = data.columns.str.replace('_Estrato ', '_')

  data['F_TIENEINTERNET'] = data['F_TIENEINTERNET'].map({'Si': 1, 'No': 0}).astype(np.int8)
  data['F_TIENELAVADORA'] = data['F_TIENELAVADORA'].map({'Si': 1, 'No': 0}).astype(np.int8)
  data['F_TIENEAUTOMOVIL'] = data['F_TIENEAUTOMOVIL'].map({'Si': 1, 'No': 0}).astype(np.int8)
  data['E_PAGOMATRICULAPROPIO'] = data['E_PAGOMATRICULAPROPIO'].map({'Si': 1, 'No': 0}).astype(np.int8)
  data['F_TIENECOMPUTADOR'] = data['F_TIENECOMPUTADOR'].map({'Si': 1, 'No': 0}).astype(np.int8)

  mapping = {
    'Ninguno': 0,
    'No sabe': 0,
    'No Aplica': 0,
    'Primaria incompleta': 1,
    'Primaria completa': 2,
    'Secundaria (Bachillerato) incompleta': 3,
    'Secundaria (Bachillerato) completa': 5,
    'Técnica o tecnológica incompleta': 7,
    'Técnica o tecnológica completa': 10,
    'Educación profesional incompleta': 13,
    'Educación profesional completa': 20,
    'Postgrado': 40
  }

  data['F_EDUCACIONMADRE'] = data['F_EDUCACIONMADRE'].map(mapping).astype(np.uint8)
  data['F_EDUCACIONPADRE'] = data['F_EDUCACIONPADRE'].map(mapping).astype(np.uint8)

  data = pd.get_dummies(data, columns=['E_PRGM_DEPARTAMENTO'], prefix='REGION', dtype=np.uint8)

  data = pd.get_dummies(data, columns=['AREA_SNIES'], dtype=np.uint8)

  return data

In [26]:
def normalize_data(data):
  data['E_HORASSEMANATRABAJA'] = (data['E_HORASSEMANATRABAJA'] - data['E_HORASSEMANATRABAJA'].min()) / (data['E_HORASSEMANATRABAJA'].max() - data['E_HORASSEMANATRABAJA'].min())
  data['F_EDUCACIONMADRE'] = (data['F_EDUCACIONMADRE'] - data['F_EDUCACIONMADRE'].min()) / (data['F_EDUCACIONMADRE'].max() - data['F_EDUCACIONMADRE'].min())
  data['F_EDUCACIONPADRE'] = (data['F_EDUCACIONPADRE'] - data['F_EDUCACIONPADRE'].min()) / (data['F_EDUCACIONPADRE'].max() - data['F_EDUCACIONPADRE'].min())

  return data

In [27]:
def clean_saber_pro_2(df):
  data = clean_data_base(df)
  data['AREA_SNIES'] = data['E_PRGM_ACADEMICO'].apply(mapper_prgm_academico_to_snies)
  data.drop('E_PRGM_ACADEMICO', axis=1, inplace=True)
  data = clean_prgm_departamento_to_region(data)
  data = clean_nulos(data)
  data = categorical_to_numeric(data)
  data = normalize_data(data)

  return data

#### Main

In [28]:
df = train.copy()
df = clean_saber_pro_2(df)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 665947 entries, 0 to 692499
Data columns (total 39 columns):
 #   Column                                                                Non-Null Count   Dtype  
---  ------                                                                --------------   -----  
 0   E_HORASSEMANATRABAJA                                                  665947 non-null  float64
 1   F_TIENEINTERNET                                                       665947 non-null  int8   
 2   F_EDUCACIONPADRE                                                      665947 non-null  float64
 3   F_TIENELAVADORA                                                       665947 non-null  int8   
 4   F_TIENEAUTOMOVIL                                                      665947 non-null  int8   
 5   E_PAGOMATRICULAPROPIO                                                 665947 non-null  int8   
 6   F_TIENECOMPUTADOR                                                     665947 non-null  in

Pasa la variable a predecir, de categorica a numérica

In [30]:
mapping = {
    'alto': 4,
    'medio-alto': 3,
    'medio-bajo': 2,
    'bajo': 1
}

df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(mapping).astype('int8')

### Test

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

Separo de mi dataset la variable a predecir

In [32]:
X = df.drop(columns=['RENDIMIENTO_GLOBAL'])
y = df.RENDIMIENTO_GLOBAL

Divido el dataset en dos subconjuntos, el 80% del dataset para entrenar el modelo y el restante para probarlo. Se usa una semilla para fijar la aleatoriedad y conseguir que la división sea reproducible.

In [33]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print(f"Train: {X_train.shape} | Test: {X_test.shape}")
print(f"Distribución train: {np.bincount(y_train)/len(y_train)}")

Train: (532757, 38) | Test: (133190, 38)
Distribución train: [0.         0.24812063 0.2501816  0.24833648 0.25336129]


Se entrena usando un RandomForestClassifier, con 100 arboles, con una profundidad de arbol de 12.
Cada hoja debe tener al menos 25 muestras.
`max_features='sqrt'` considera la raiz cuadrada del numero de caracteristicas, para dividir el nodo. Esto fomentan la diversidad entre los nodos.
`oob_score=True` habilita el cálculo de la puntuación de "out-of-bag", lo que permite evaluar la precisión del modelo sin necesidad de un conjunto de validación separado.

#### Evaluación del modelo:
  - Train Accuracy: medida de ajuste (qué tan bien el modelo aprende los datos de entrenamiento).
  - Test Accuracy: medida de generalización (qué tan bien predice en datos nuevos).
  - OOB Score: estimación interna de la accuracy (sin usar datos de test), basada en muestras "fuera de la bolsa".
  - Overfitting gap = Train Acc - Test Acc: • ≈ 0 → modelo bien balanceado (ideal) • > 0.05 → posible sobreajuste (modelo memoriza ruido) • < 0 → posible subajuste o fuga de datos (train_acc < test_acc)

In [34]:
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=12,
    min_samples_split=50,
    min_samples_leaf=25,
    max_features='sqrt',
    n_jobs=-1,
    random_state=42,
    oob_score=True
)

rf.fit(X_train, y_train)

train_acc = rf.score(X_train, y_train)
test_acc = rf.score(X_test, y_test)
oob_score = rf.oob_score_

print(" Resultados Random Forest:")
print(f"   Train Accuracy: {train_acc:.4f}")
print(f"   Test Accuracy:  {test_acc:.4f}")
print(f"   OOB Score:      {oob_score:.4f}")
print(f"   Overfitting:    {train_acc - test_acc:.4f} (ideal < 0.05)") # Probamos Sobreajuste

 Resultados Random Forest:
   Train Accuracy: 0.4207
   Test Accuracy:  0.4021
   OOB Score:      0.4003
   Overfitting:    0.0186 (ideal < 0.05)
